In [ ]:
# Cell 1: Robust Library Installation
# Step 1: Clean up any potentially conflicting versions
print("--> Uninstalling existing versions...")
!pip uninstall -y transformers accelerate datasets torch torchvision

# Step 2: Reinstall a stable, compatible set of the core libraries
print("\n--> Reinstalling core libraries...")
!pip install transformers accelerate datasets torch torchvision

# Step 3: Install the remaining libraries
print("\n--> Installing other required libraries...")
!pip install scikit-learn pandas imbalanced-learn -q

print("\n✅ All libraries have been reinstalled.")

--> Uninstalling existing versions...
Found existing installation: transformers 4.55.1
Uninstalling transformers-4.55.1:
  Successfully uninstalled transformers-4.55.1
Found existing installation: accelerate 1.10.0
Uninstalling accelerate-1.10.0:
  Successfully uninstalled accelerate-1.10.0
Found existing installation: datasets 4.0.0
Uninstalling datasets-4.0.0:
  Successfully uninstalled datasets-4.0.0
Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124

--> Reinstalling core libraries...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 123.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494

In [ ]:
# Cell 2: Check Library Versions
# This output should be noted for the 'reproducibility' section of the paper
!pip freeze | grep -E "transformers|torch|scikit-learn|datasets"

datasets==4.0.0
scikit-learn==1.6.1
sentence-transformers==5.1.0
tensorflow-datasets==4.9.9
torch==2.8.0
torchao==0.10.0
torchaudio @ https://download.pytorch.org/whl/cu124/torchaudio-2.6.0%2Bcu124-cp311-cp311-linux_x86_64.whl
torchdata==0.11.0
torchsummary==1.5.1
torchtune==0.6.1
torchvision==0.23.0
transformers==4.55.2
vega-datasets==0.9.0


In [ ]:
# Cell 3: Connect to Drive, Setup Paths & Initialize
from google.colab import drive
import pandas as pd
import numpy as np
import pickle
import os
import json
import torch
from torch import nn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from transformers import (
    AutoTokenizer,
    AutoConfig, # <-- Import AutoConfig
    Trainer,
    TrainingArguments,
    BertPreTrainedModel,
    BertModel
)
from transformers.modeling_outputs import SequenceClassifierOutput
from datasets import Dataset

# Mount Google Drive
drive.mount('/content/drive')

# --- KEY SETTINGS ---
SEED = 42
GDRIVE_PATH = '/content/drive/MyDrive/eecsi_revise/'
MODEL_NAME = "indolem/indobertweet-base-uncased"

# --- NEW: Define a dedicated path for MTL results ---
MTL_RESULTS_PATH = os.path.join(GDRIVE_PATH, 'mtl_indobertweet_results/')
os.makedirs(MTL_RESULTS_PATH, exist_ok=True)

# Set seed for reproducibility across all libraries
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

print(f"✅ Setup complete. MTL checkpoints and results will be saved to: {MTL_RESULTS_PATH}")

Mounted at /content/drive
✅ Setup complete. MTL checkpoints and results will be saved to: /content/drive/MyDrive/eecsi_revise/mtl_indobertweet_results/


In [ ]:
# Cell 4: Load and Prepare Data for MTL
file_path_csv = os.path.join(GDRIVE_PATH, 'final_golden_dataset_eecsi.csv')
df = pd.read_csv(file_path_csv)

# --- CRITICAL: Use relevant data only ---
relevant_df = df[df['aspect'] != 'Irrelevant'].copy()
print(f"Using {len(relevant_df)} relevant rows for Multi-Task Learning.")

Using 2037 relevant rows for Multi-Task Learning.


In [ ]:
# Cell 5: Define the Multi-Task Learning Model Architecture
from torch import nn
from transformers import BertPreTrainedModel, BertModel
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import AutoConfig # Make sure AutoConfig is imported

class IndoBERTweet_MTL(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        # Store the number of labels for each task from the model's config
        self.num_aspect_labels = config.num_aspect_labels
        self.num_sentiment_labels = config.num_sentiment_labels

        # Shared Body: The core IndoBERTweet model
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

        # Head 1: The Aspect Classifier
        self.aspect_classifier = nn.Linear(config.hidden_size, self.num_aspect_labels)

        # Head 2: The Sentiment Classifier
        self.sentiment_classifier = nn.Linear(config.hidden_size, self.num_sentiment_labels)

        # Initialize the weights of the new layers
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        aspect_labels=None,    # Custom label name for aspects
        sentiment_labels=None, # Custom label name for sentiments
        return_dict=None,
        **kwargs,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        # Pass inputs through the shared BERT body
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            return_dict=return_dict,
        )

        # Use the pooled output for classification
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)

        # Get logits (raw predictions) from both heads
        aspect_logits = self.aspect_classifier(pooled_output)
        sentiment_logits = self.sentiment_classifier(pooled_output)

        total_loss = 0
        loss_fct = nn.CrossEntropyLoss()

        # --- Calculate the Combined Loss ---
        # 1. Aspect Loss (if labels are provided)
        if aspect_labels is not None:
            loss_aspect = loss_fct(aspect_logits.view(-1, self.num_aspect_labels), aspect_labels.view(-1))
            total_loss += loss_aspect

        # 2. Sentiment Loss (if labels are provided)
        if sentiment_labels is not None:
            loss_sentiment = loss_fct(sentiment_logits.view(-1, self.num_sentiment_labels), sentiment_labels.view(-1))
            total_loss += loss_sentiment

        # The Trainer API expects a specific output format
        return SequenceClassifierOutput(
            loss=total_loss,
            logits=(aspect_logits, sentiment_logits), # Return both logits as a tuple
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

print("✅ Custom MTL model class 'IndoBERTweet_MTL' defined.")

✅ Custom MTL model class 'IndoBERTweet_MTL' defined.


In [ ]:
# Cell 6: Prepare Helper Functions & Classes for MTL

# 1. Custom PyTorch Dataset for MTL
class MTLDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, aspect_labels, sentiment_labels):
        self.encodings = encodings
        self.aspect_labels = aspect_labels
        self.sentiment_labels = sentiment_labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # This dataset returns two different labels
        item['aspect_labels'] = torch.tensor(self.aspect_labels[idx])
        item['sentiment_labels'] = torch.tensor(self.sentiment_labels[idx])
        return item

    def __len__(self):
        return len(self.aspect_labels)

# 2. Custom compute_metrics function for MTL
def compute_metrics_mtl(p):
    # p.predictions is now a tuple of two arrays (aspect_preds, sentiment_preds)
    aspect_preds = np.argmax(p.predictions[0], axis=1)
    sentiment_preds = np.argmax(p.predictions[1], axis=1)

    # p.label_ids is also a tuple of two arrays
    aspect_labels = p.label_ids[0]
    sentiment_labels = p.label_ids[1]

    # Calculate metrics for each task separately
    aspect_report = classification_report(aspect_labels, aspect_preds, output_dict=True, zero_division=0)
    sentiment_report = classification_report(sentiment_labels, sentiment_preds, output_dict=True, zero_division=0)

    # Return a dictionary with metrics for both tasks
    return {
        "aspect_macro_f1": aspect_report["macro avg"]["f1-score"],
        "sentiment_macro_f1": sentiment_report["macro avg"]["f1-score"]
    }

print("✅ MTL-specific helper functions and classes are ready.")

✅ MTL-specific helper functions and classes are ready.


In [ ]:
# Cell 7: Run the 5-Fold Cross-Validation for MTL
from transformers import AutoConfig # Make sure AutoConfig is imported

fold_results = []
X = relevant_df['cleaned_text']
y_aspect = relevant_df['aspect']
y_sentiment = relevant_df['sentiment']

# --- Create Label Mappings for BOTH tasks ---
aspect_labels_list = np.array(sorted(y_aspect.unique()))
sentiment_labels_list = np.array(sorted(y_sentiment.unique()))

aspect_label2id = {label: i for i, label in enumerate(aspect_labels_list)}
sentiment_label2id = {label: i for i, label in enumerate(sentiment_labels_list)}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

# We stratify by aspect as it is the more complex task with more classes
for i, (train_index, test_index) in enumerate(skf.split(X, y_aspect)):
    print(f"--- Running Fold {i+1}/5 ---")

    train_df = relevant_df.iloc[train_index]
    test_df = relevant_df.iloc[test_index]

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    train_encodings = tokenizer(list(train_df['cleaned_text']), truncation=True, padding=True, max_length=128)
    test_encodings = tokenizer(list(test_df['cleaned_text']), truncation=True, padding=True, max_length=128)

    # Create two sets of labels for training and testing
    train_aspect_labels = [aspect_label2id[label] for label in train_df['aspect']]
    train_sentiment_labels = [sentiment_label2id[label] for label in train_df['sentiment']]
    test_aspect_labels = [aspect_label2id[label] for label in test_df['aspect']]
    test_sentiment_labels = [sentiment_label2id[label] for label in test_df['sentiment']]

    train_dataset = MTLDataset(train_encodings, train_aspect_labels, train_sentiment_labels)
    test_dataset = MTLDataset(test_encodings, test_aspect_labels, test_sentiment_labels)

    # Get model config and add our custom parameters for the two heads
    config = AutoConfig.from_pretrained(MODEL_NAME)
    config.num_aspect_labels = len(aspect_labels_list)
    config.num_sentiment_labels = len(sentiment_labels_list)

    # Initialize our custom MTL model
    model = IndoBERTweet_MTL.from_pretrained(MODEL_NAME, config=config)

    if i == 0:
        print(f"Model Parameters: {model.num_parameters()/1e6:.2f}M")

    training_args = TrainingArguments(
        output_dir=os.path.join(MTL_RESULTS_PATH, f'fold_{i+1}'),
        num_train_epochs=5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        learning_rate=2e-5,
        eval_strategy="epoch",
        save_strategy="epoch",
        # We need to pick one primary metric to decide the "best" model checkpoint
        load_best_model_at_end=True,
        metric_for_best_model="eval_aspect_macro_f1",
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        seed=SEED,
        # This is required because our dataset has custom column names ('aspect_labels', etc.)
        remove_unused_columns=False,
    )

    # Use the standard Trainer, not the weighted one
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics_mtl,
    )

    trainer.train()

    eval_results = trainer.evaluate()
    fold_results.append(eval_results)
    print(f"Fold {i+1} complete. Evaluation results: {eval_results}")

print("\n✅ 5-fold cross-validation process for MTL finished.")

--- Running Fold 1/5 ---


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

Epoch,Training Loss,Validation Loss,Aspect Macro F1,Sentiment Macro F1
1,1.757300,1.760434,0.569436,0.653401
2,1.151900,1.327015,0.697740,0.746647
3,0.844600,1.191917,0.760953,0.749282
4,0.756700,1.135448,0.779564,0.757867
5,0.611600,1.112478,0.779895,0.785164


Fold 4 complete. Evaluation results: {'eval_loss': 1.112478494644165, 'eval_aspect_macro_f1': 0.7798949014549331, 'eval_sentiment_macro_f1': 0.7851641293959969, 'eval_runtime': 1.7269, 'eval_samples_per_second': 235.683, 'eval_steps_per_second': 15.056, 'epoch': 5.0}
--- Running Fold 5/5 ---


Some weights of IndoBERTweet_MTL were not initialized from the model checkpoint at indolem/indobertweet-base-uncased and are newly initialized: ['aspect_classifier.bias', 'aspect_classifier.weight', 'sentiment_classifier.bias', 'sentiment_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Aspect Macro F1,Sentiment Macro F1
1,1.938400,1.799620,0.568435,0.673141
2,1.223800,1.344574,0.697131,0.746616
3,0.969000,1.185501,0.744486,0.755606
4,0.802000,1.118918,0.756344,0.779488
5,0.587700,1.099494,0.754658,0.768002


Fold 5 complete. Evaluation results: {'eval_loss': 1.1189175844192505, 'eval_aspect_macro_f1': 0.7563441836957207, 'eval_sentiment_macro_f1': 0.7794881142249563, 'eval_runtime': 1.5002, 'eval_samples_per_second': 271.299, 'eval_steps_per_second': 17.331, 'epoch': 5.0}

✅ 5-fold cross-validation process for MTL finished.


In [ ]:
# Cell 8: Aggregate and Display Final Results

# Extract scores for both tasks from the results list
aspect_f1_scores = [result['eval_aspect_macro_f1'] for result in fold_results]
sentiment_f1_scores = [result['eval_sentiment_macro_f1'] for result in fold_results]

# Calculate mean and std for the Aspect Detection task
mean_aspect_f1 = np.mean(aspect_f1_scores)
std_aspect_f1 = np.std(aspect_f1_scores)

# Calculate mean and std for the Sentiment Classification task
mean_sentiment_f1 = np.mean(sentiment_f1_scores)
std_sentiment_f1 = np.std(sentiment_f1_scores)

print("--- Final Aggregated Results (5-Fold CV) for MTL Model ---")
print(f"Aspect Detection Macro F1-Score = {mean_aspect_f1:.4f} ± {std_aspect_f1:.4f}")
print(f"Sentiment Classification Macro F1-Score = {mean_sentiment_f1:.4f} ± {std_sentiment_f1:.4f}")

--- Final Aggregated Results (5-Fold CV) for MTL Model ---
Aspect Detection Macro F1-Score = 0.7749 ± 0.0154
Sentiment Classification Macro F1-Score = 0.7846 ± 0.0138


In [ ]:
# Cell 9: Save Results to Google Drive
final_results = {
    'model': 'IndoBERTweet (MTL)',
    'mean_aspect_macro_f1': mean_aspect_f1,
    'std_dev_aspect_macro_f1': std_aspect_f1,
    'mean_sentiment_macro_f1': mean_sentiment_f1,
    'std_dev_sentiment_macro_f1': std_sentiment_f1,
    'results_per_fold': fold_results
}

results_file_path = os.path.join(MTL_RESULTS_PATH, 'results_mtl.json')
with open(results_file_path, 'w') as f:
    json.dump(final_results, f, indent=4)

print(f"\n✅ Final results for MTL have been saved to: '{results_file_path}'")


✅ Final results for MTL have been saved to: '/content/drive/MyDrive/eecsi_revise/mtl_indobertweet_results/results_mtl.json'
